# PLS 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.api import qqplot
from statsmodels.formula.api import ols #for linear regression
import seaborn as sns #for heatmap
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale 

In [2]:
data=pd.read_csv("Automobile_data.csv")

In [3]:
data.head()

make fuel_type aspiration num_of_doors   body_style drive_wheels  \
0  alfa-romero       gas        std          two  convertible          rwd   
1  alfa-romero       gas        std          two  convertible          rwd   
2  alfa-romero       gas        std          two    hatchback          rwd   
3         audi       gas        std         four        sedan          fwd   
4         audi       gas        std         four        sedan          4wd   

  engine_location  wheel_base  length  width   ...     engine_size  \
0           front        88.6   168.8   64.1   ...             130   
1           front        88.6   168.8   64.1   ...             130   
2           front        94.5   171.2   65.5   ...             152   
3           front        99.8   176.6   66.2   ...             109   
4           front        99.4   176.6   66.4   ...             136   

   fuel_system  bore stroke  compression_ratio horsepower  peak_rpm  city_mpg  \
0         mpfi  3.47   2.68                9.0      111.0    5000.0        21   
1         mpfi  3.47   2.68                9.0      111.0    5000.0        21   
2         mpfi  2.68   3.47                9.0      154.0    5000.0        19   
3         mpfi  3.19   3.40               10.0      102.0    5500.0        24   
4         mpfi  3.19   3.40                8.0      115.0    5500.0        18   

   highway_mpg    price  
0           27  13495.0  
1           27  16500.0  
2           26  16500.0  
3           30  13950.0  
4           22  17450.0  

[5 rows x 24 columns]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 24 columns):
make                 205 non-null object
fuel_type            205 non-null object
aspiration           205 non-null object
num_of_doors         203 non-null object
body_style           205 non-null object
drive_wheels         205 non-null object
engine_location      205 non-null object
wheel_base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb_weight          205 non-null int64
engine_type          205 non-null object
num_of_cylinders     205 non-null object
engine_size          205 non-null int64
fuel_system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression_ratio    205 non-null float64
horsepower           203 non-null float64
peak_rpm             203 non-null float64
city_mpg             205 no

In [5]:
data.isnull().any()

make                 False
fuel_type            False
aspiration           False
num_of_doors          True
body_style           False
drive_wheels         False
engine_location      False
wheel_base           False
length               False
width                False
height               False
curb_weight          False
engine_type          False
num_of_cylinders     False
engine_size          False
fuel_system          False
bore                  True
stroke                True
compression_ratio    False
horsepower            True
peak_rpm              True
city_mpg             False
highway_mpg          False
price                 True
dtype: bool

In [6]:
data=data.dropna()

In [7]:
#we see that we have many correlated variables
plt.figure(figsize=(9, 7))
sns.heatmap(data.corr().abs().round(2), annot=True)
plt.show()

In [8]:
data_numeric = data.select_dtypes(exclude="object")
data_object = data.select_dtypes(include="object")

In [9]:
data_numeric.var()==0

wheel_base           False
length               False
width                False
height               False
curb_weight          False
engine_size          False
bore                 False
stroke               False
compression_ratio    False
horsepower           False
peak_rpm             False
city_mpg             False
highway_mpg          False
price                False
dtype: bool

In [32]:
for i in data_object.columns:
    print(data_object[i].unique())

['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'mercedes-benz' 'mercury' 'mitsubishi' 'nissan' 'peugot'
 'plymouth' 'porsche' 'saab' 'subaru' 'toyota' 'volkswagen' 'volvo']
['gas' 'diesel']
['std' 'turbo']
['two' 'four']
['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
['rwd' 'fwd' '4wd']
['front' 'rear']
['dohc' 'ohcv' 'ohc' 'l' 'ohcf']
['four' 'six' 'five' 'three' 'twelve' 'eight']
['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' 'idi' 'spdi']


In [11]:
data_dummy = pd.get_dummies(data_object,prefix=data_object.columns,drop_first=True)

In [12]:
data_dummy.head()

make_audi  make_bmw  make_chevrolet  make_dodge  make_honda  make_isuzu  \
0          0         0               0           0           0           0   
1          0         0               0           0           0           0   
2          0         0               0           0           0           0   
3          1         0               0           0           0           0   
4          1         0               0           0           0           0   

   make_jaguar  make_mazda  make_mercedes-benz  make_mercury  \
0            0           0                   0             0   
1            0           0                   0             0   
2            0           0                   0             0   
3            0           0                   0             0   
4            0           0                   0             0   

         ...         num_of_cylinders_four  num_of_cylinders_six  \
0        ...                             1                     0   
1        ...                             1                     0   
2        ...                             0                     1   
3        ...                             1                     0   
4        ...                             0                     0   

   num_of_cylinders_three  num_of_cylinders_twelve  fuel_system_2bbl  \
0                       0                        0                 0   
1                       0                        0                 0   
2                       0                        0                 0   
3                       0                        0                 0   
4                       0                        0                 0   

   fuel_system_idi  fuel_system_mfi  fuel_system_mpfi  fuel_system_spdi  \
0                0                0                 1                 0   
1                0                0                 1                 0   
2                0                0                 1                 0   
3                0                0                 1                 0   
4                0                0                 1                 0   

   fuel_system_spfi  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  

[5 rows x 45 columns]

In [13]:
data = data_numeric.join(data_dummy)

In [14]:
Y=data.price
X = scale(data.drop(columns="price"))

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 204
Data columns (total 59 columns):
wheel_base                 193 non-null float64
length                     193 non-null float64
width                      193 non-null float64
height                     193 non-null float64
curb_weight                193 non-null int64
engine_size                193 non-null int64
bore                       193 non-null float64
stroke                     193 non-null float64
compression_ratio          193 non-null float64
horsepower                 193 non-null float64
peak_rpm                   193 non-null float64
city_mpg                   193 non-null int64
highway_mpg                193 non-null int64
price                      193 non-null float64
make_audi                  193 non-null uint8
make_bmw                   193 non-null uint8
make_chevrolet             193 non-null uint8
make_dodge                 193 non-null uint8
make_honda                 193 non-null uint8
m

In [16]:
# split 75% train and 25% test
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [17]:
pls = PLSRegression(10)
pls=pls.fit(x_train,y_train)

In [18]:
pred=pls.predict(x_test)

In [19]:
#R-square
pls_r2=pls.score(x_test,y_test)
pls_r2

0.9014636210257868

In [20]:
r2_score(y_test,pred)

0.9014636210257868

In [21]:
coef=pd.DataFrame(pls.coef_)
coef

0
0   9.345760e+02
1  -2.036375e+02
2   8.538516e+02
3  -5.359844e+02
4   1.332016e+03
5   1.803691e+03
6  -5.042674e+02
7  -4.051068e+02
8  -1.144462e+02
9   7.933700e+02
10  2.656985e+01
11 -1.796915e+02
12 -3.685677e+01
13  8.029595e+02
14  1.113627e+03
15  1.003278e+02
16 -4.572696e+02
17  9.203783e+01
18 -3.558878e+02
19  7.483069e+02
20 -9.185017e+01
21  1.347521e+03
22  2.980059e-27
23 -7.054192e+02
24 -2.883093e+02
25 -3.731307e+02
26 -2.925831e+02
27  6.143685e+02
28  3.836714e+02
29 -5.802100e+02
30 -6.804106e+02
31  1.644311e+01
32 -7.797443e+01
33 -1.165665e+02
34  8.132190e+02
35 -4.211497e+02
36 -3.468086e+02
37 -8.899774e+02
38 -9.816801e+02
39 -6.031747e+02
40 -4.119596e+02
41  5.231788e+02
42  1.453014e+03
43 -3.225267e+02
44  7.957901e+01
45  1.114357e+02
46 -1.587429e+02
47 -8.934824e+02
48 -6.506620e+02
49 -7.714778e+02
50  1.003278e+02
51  2.980059e-27
52  1.656752e+02
53  1.165665e+02
54 -1.088388e+02
55 -1.349062e+01
56  1.252972e+02
57  2.980059e-27

In [22]:
sc=pd.DataFrame(pls.x_scores_)
sc.head(10)

0         1         2         3         4         5         6  \
0 -3.433320 -0.176857  0.824141 -0.895387 -1.218353  1.226609 -1.648470   
1 -4.330832  1.790565  0.361683 -0.131705  0.448625 -0.621482  0.189363   
2 -2.479803 -0.254193 -3.048170  2.686994  0.461518  1.173217 -0.324596   
3  1.645651 -0.418485 -0.183017 -0.586288 -1.808144 -1.483844  0.272358   
4 -1.851599  0.724931 -0.570809 -0.566040 -0.675670 -1.620077  0.169243   
5 -0.473723 -0.997241  0.026305  0.415180 -0.597346 -0.121187  0.423153   
6 -1.992775  0.182993 -0.386001 -0.365725 -0.506689 -1.108859 -0.229451   
7 -3.698946  1.414302  0.196178 -0.267388  0.231590 -0.970890  0.110707   
8  0.867911  0.591211  2.148038  2.620782 -2.305301 -0.341413  0.721079   
9  2.982918  1.123054 -0.502065 -0.946337  0.326016  0.445819 -0.595535   

          7         8         9  
0  1.062604  0.660011 -0.322489  
1  0.185083 -0.890425 -0.237913  
2 -0.986013 -0.518027 -0.273686  
3  1.195819 -1.182685 -0.042218  
4  1.604793 -0.950464  0.658175  
5 -0.557978  0.326630  1.087279  
6 -0.852513 -1.279661 -0.343633  
7  0.158795 -1.137566 -0.561874  
8  1.403980 -0.163382 -0.046572  
9 -0.227821 -0.768800  0.550725

In [23]:
w=pd.DataFrame(pls.x_weights_)
w

0             1             2             3             4  \
0   2.070780e-01 -1.492010e-01 -2.150375e-01 -1.993302e-02 -1.534406e-02   
1   2.381401e-01 -1.592032e-01 -9.904991e-02  5.564771e-03 -9.131280e-02   
2   2.551469e-01 -3.072044e-02 -1.268799e-01 -8.927422e-03  4.673372e-02   
3   5.729298e-02 -2.055800e-01 -2.273391e-01  7.737588e-02 -2.278723e-01   
4   2.932906e-01 -3.764436e-02 -5.441211e-02 -2.839187e-02  9.525768e-02   
5   3.023182e-01  1.539489e-01 -1.768368e-02 -8.146868e-02  1.437552e-01   
6   1.791129e-01 -1.140981e-01  4.711794e-02  3.505179e-02 -2.545394e-01   
7   4.331477e-02  4.141274e-02  1.167296e-01  3.061790e-01  5.933644e-02   
8   2.819958e-02  2.766006e-02 -1.587375e-01  3.913964e-01  1.284251e-01   
9   2.721048e-01  8.132205e-02  1.662390e-01 -7.761177e-02  4.351803e-02   
10 -3.660344e-02  5.508300e-02  1.618473e-01 -1.171507e-01 -1.016209e-01   
11 -2.373486e-01  8.472596e-02 -1.629877e-01  9.739456e-02  1.194931e-01   
12 -2.420302e-01  9.935074e-02 -1.404025e-01  8.286745e-02  1.172091e-01   
13  4.356494e-02  1.454404e-02 -2.893874e-02 -3.556486e-02  1.429088e-01   
14  8.260915e-02  1.534675e-01 -1.455521e-01 -1.688535e-01  3.831214e-02   
15 -3.158474e-02  1.335841e-01 -3.486526e-02  7.272956e-02 -1.049513e-01   
16 -5.635955e-02  4.700370e-02  3.115408e-02  1.678519e-02 -9.523923e-02   
17 -6.487524e-02  1.069716e-01 -2.996614e-02 -9.695661e-03 -9.572342e-02   
18 -2.531142e-02 -8.645260e-02  8.068173e-02  6.875294e-02 -3.046760e-02   
19  1.118170e-01  1.382988e-01 -1.134984e-01 -1.522443e-01  1.061910e-01   
20 -4.822463e-02  5.915329e-02 -4.684160e-02  9.344515e-02 -2.614899e-01   
21  1.840145e-01  3.053312e-01 -3.081923e-01 -7.663304e-02  4.003006e-02   
22  7.103813e-33  5.658399e-32 -7.326316e-32 -1.493157e-31  2.292744e-31   
23 -3.340402e-02 -2.290213e-02  1.421109e-01  1.889943e-02  4.931190e-02   
24 -4.277209e-02 -1.337169e-01  2.021228e-01 -1.866087e-02  3.429148e-01   
25  3.190015e-02 -2.851118e-01 -1.097397e-01 -2.700548e-02  6.407302e-02   
26 -2.081183e-02  1.096850e-02  4.456947e-02 -1.584423e-02  3.708512e-02   
27  1.148282e-01  3.557201e-01  1.471825e-02  9.874189e-03 -1.187992e-01   
28  1.766716e-02  3.791008e-02 -1.064569e-01 -1.316985e-01 -1.671657e-01   
29 -6.297199e-02 -1.255233e-01  6.413989e-02 -1.143919e-03 -2.000820e-01   
30 -6.844188e-02 -1.128601e-01  1.249378e-01  1.358678e-01 -3.407913e-03   
31 -4.105161e-02 -2.637572e-02 -9.864336e-03  1.367475e-01  1.180945e-01   
32  6.459270e-02 -1.310748e-01 -9.522698e-03 -3.333873e-02 -4.052467e-02   
33 -4.259737e-02 -1.408086e-02  1.609493e-01 -3.785547e-01 -1.611985e-01   
34  7.674787e-02 -3.629804e-02 -8.608320e-02  3.188400e-02  3.132800e-01   
35 -6.123438e-02  2.161389e-01  2.000362e-01  2.149566e-02  2.364127e-02   
36  2.295322e-02  6.131355e-02  2.081933e-01  1.929366e-01 -8.947021e-02   
37 -1.091098e-01  1.096840e-01  1.831237e-01  3.174478e-02 -3.187769e-02   
38  5.880462e-02 -1.133129e-01 -1.098911e-01  1.259034e-01 -1.880805e-01   
39  3.227999e-03 -1.905309e-01 -3.238387e-02 -5.277354e-02  1.257070e-02   
40 -2.232777e-01  3.742325e-02 -1.420264e-02 -2.266943e-02  4.939873e-02   
41  2.321378e-01 -7.888842e-03 -1.364799e-02  2.928905e-03 -3.592011e-03   
42  1.166008e-01  3.672610e-01 -1.199427e-02 -3.939355e-02  9.096744e-03   
43  2.005791e-02 -2.278496e-01 -1.158856e-01 -1.956152e-03  2.672645e-02   
44 -1.155869e-01  1.435994e-01 -1.322889e-01  6.010689e-02 -4.500086e-02   
45 -6.013539e-03  4.742184e-02  5.316672e-02 -1.839896e-02 -1.783287e-01   
46  1.172973e-01 -7.374002e-02  2.405229e-01 -1.603252e-02  2.157695e-01   
47  9.691241e-02  3.939997e-02  4.940983e-02  3.507904e-01 -3.017575e-01   
48 -2.332012e-01 -1.601031e-01 -1.553792e-01 -1.578722e-01  2.659378e-03   
49  1.593660e-01 -1.818558e-03  3.429474e-01  1.648437e-01 -7.705862e-03   
50 -3.158474e-02  1.335841e-01 -3.486526e-02  7.272956e-02 -1.049513e-01   
51  7.103813e-33  5.658399e-32 -7.326316e-32 -1.493157e-31  2.2

In [24]:
var_xcores=np.var(pls.x_scores_,axis=0)
var_xcores

array([10.23541948,  2.55232414,  2.68955472,  1.73187538,  1.05178474,
        1.23728727,  1.34266882,  1.17193835,  0.74222052,  0.57954627])

In [25]:
total_var=np.sum(np.var(x_train,axis=0))
total_var

54.68950987170588

In [26]:
X_Variance=pd.DataFrame((var_xcores/total_var).round(3), columns={"X_Variance"}).reset_index()

In [27]:
X_Variance = X_Variance.rename(columns={'index': 'Latent_Factors'})
X_Variance['Cummulative_X_Variance'] = X_Variance['X_Variance'].cumsum().round(3)
X_Variance

Latent_Factors  X_Variance  Cummulative_X_Variance
0               0       0.187                   0.187
1               1       0.047                   0.234
2               2       0.049                   0.283
3               3       0.032                   0.315
4               4       0.019                   0.334
5               5       0.023                   0.357
6               6       0.025                   0.382
7               7       0.021                   0.403
8               8       0.014                   0.417
9               9       0.011                   0.428

In [28]:
plt.plot(X_Variance.Latent_Factors,X_Variance.Cummulative_X_Variance)

# I manged to do this for train only I cannot use x_score and Y_loadings from pls model to predict for test set, I need more time to understan

In [29]:
r2_sum = 0
for i in range(0,10):
    Y_pred1=np.dot(pls.x_scores_[:,i].reshape(-1,1),pls.y_loadings_[:,i].reshape(-1,1).T)*y_train.std()+y_train.mean()
    r2_sum += round(r2_score(y_train,Y_pred1),3) 
    print('R2 for %d component: %g' %(i+1,round(r2_score(y_train,Y_pred1),3)))
print('R2 for all components (): %g' %r2_sum) #Sum of above
print('R2 for all components (): %g' %round(r2_score(y_test,pred),3)) #Calcuted from PLSRegression's 'predict' function.

R2 for 1 component: 0.83
R2 for 2 component: 0.082
R2 for 3 component: 0.023
R2 for 4 component: 0.015
R2 for 5 component: 0.009
R2 for 6 component: 0.005
R2 for 7 component: 0.002
R2 for 8 component: 0.001
R2 for 9 component: 0.001
R2 for 10 component: 0.002
R2 for all components (): 0.97
R2 for all components (): 0.901


# Linear Regression

In [30]:
lin_reg=LinearRegression().fit(X,Y)
pr_l=lin_reg.predict(x_test)
Linear_r2=r2_score(y_test,pr_l)
Linear_r2

0.9631438719699826

In [31]:
#comparison
(pls_r2,Linear_r2)

(0.9014636210257868, 0.9631438719699826)